In [30]:
import src.config
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.now().date()).floor('h')
fetch_data_from = current_date - timedelta(days=1)
fetch_data_to = current_date

In [33]:
# simulate a call to a datawerehouse
from src.load import load_raw_data
from src.paths import YELLOW, GREEN, FHV, FHVHV

def fetch_batch_raw_data(from_date: datetime, to_date:datetime) ->pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)
    """
    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)
    print(f'{from_date=}, {to_date_=}')
    
    # download 2 files from website
    for path in [YELLOW]:#, GREEN, FHV, FHVHV]:
        rides = load_raw_data(path,year=from_date_.year, months=from_date_.month)
        rides = rides[rides.pickup_datetime >= from_date_]
        rides_2 = load_raw_data(path,year=to_date_.year, months=to_date_.month)
        rides_2 = rides_2[rides_2.pickup_datetime < to_date_]
    
    

    rides = pd.concat([rides, rides_2])

    # shift the data to pretend this is recent data
    rides['pickup_datetime'] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [ ]:


from src.load import load_raw_data
from src.paths import YELLOW

data = load_raw_data(PATH=YELLOW, year=2022, months=[1])


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
File 2022-01 was already in local storage


In [1]:
data

NameError: name 'data' is not defined

In [35]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

from_date=Timestamp('2024-11-03 00:00:00'), to_date_=Timestamp('2023-11-06 00:00:00')
File 2023-11 was already in local storage
File 2023-11 was already in local storage


In [44]:
def get_dot_position(pattern):
    dot='.'
    dot_position_list = [position for position,char in enumerate(pattern) if char==dot]

    return dot_position_list

def match_word(word,pattern):
    is_match = False
    if len(word) != len(pattern):
        return is_match

    dot_position_list = get_dot_position(pattern)

    
    for position in range(len(word)):
        if position not in dot_position_list:
            if word[position] == pattern[position]:
                is_match = True
            else:
                is_match = False
    
    return is_match



def wordsPatternsMatch(words, patterns):
    match_list = []

    for pattern in patterns:
        for word in words:
            if match_word(word,pattern):
                if pattern not in match_list:
                    match_list.append(pattern)
    return match_list

In [49]:
def bracket_lists(str):

    bracket_list = [(pos,char) for pos,char in enumerate(str) if char=='{' or char =='}']
    parent_list = [(pos,char) for pos,char in enumerate(str) if char=='(' or char == ')']
    colch_list = [(pos,char) for pos,char in enumerate(str) if char=='[' or char == ']']

    return bracket_list,parent_list,colch_list

In [52]:
def isValid(s: str):
    bracket_map = {
        '}':'{',
        ']': '[',
        ')': '('
    }
    stack = []

    for char in s:
        if char in bracket_map.values():
            stack.append(char)
        
        elif char in bracket_map.keys():
            if bracket_map[char] != stack.pop() or stack == []:
                return False

    return len(stack) == 0

# Exemplo de uso:
s = "a{b(c)d}e"
print(isValid(s))  # Output: True

False


In [56]:
import heapq

def min_cost_to_destination(grid, amount):
    n, m = len(grid), len(grid[0])  # Dimensões da grade
    directions = [(1, 0), (0, 1), (1, 1)]  # Movimentos: baixo, direita, diagonal

    # Matriz para armazenar o menor custo para cada célula
    min_cost = [[float('inf')] * m for _ in range(n)]
    min_cost[0][0] = grid[0][0]

    # Fila de prioridade para explorar caminhos de menor custo primeiro
    heap = [(grid[0][0], 0, 0)]  # (custo acumulado, x, y)

    while heap:
        current_cost, x, y = heapq.heappop(heap)

        # Se alcançamos o destino
        if (x, y) == (n - 1, m - 1):
            remaining_budget = amount - current_cost
            return remaining_budget if remaining_budget >= 0 else -1

        # Explorar os movimentos possíveis
        for dx, dy in directions:
            nx, ny = x + dx, y + dy

            # Verificar se está dentro dos limites da matriz
            if 0 <= nx < n and 0 <= ny < m:
                new_cost = current_cost + grid[nx][ny]

                # Se encontramos um caminho de menor custo para a célula (nx, ny)
                if new_cost < min_cost[nx][ny]:
                    min_cost[nx][ny] = new_cost
                    heapq.heappush(heap, (new_cost, nx, ny))

    # Se não alcançamos o destino dentro do orçamento
    return -1


In [58]:
from collections import deque

def min_cost_to_destination(grid, amount):
    n, m = len(grid), len(grid[0])  # Dimensões da grade
    directions = [(1, 0), (0, 1), (1, 1)]  # Movimentos: baixo, direita, diagonal

    # Matriz para armazenar o menor custo para cada célula
    min_cost = [[float('inf')] * m for _ in range(n)]
    min_cost[0][0] = grid[0][0]

    # Fila de busca em largura com controle do custo acumulado
    queue = deque([(grid[0][0], 0, 0)])  # (custo acumulado, x, y)

    while queue:
        current_cost, x, y = queue.popleft()

        # Se alcançamos o destino
        if (x, y) == (n - 1, m - 1):
            remaining_budget = amount - current_cost
            return remaining_budget if remaining_budget >= 0 else -1

        # Explorar os movimentos possíveis
        for dx, dy in directions:
            nx, ny = x + dx, y + dy

            # Verificar se está dentro dos limites da matriz
            if 0 <= nx < n and 0 <= ny < m:
                new_cost = current_cost + grid[nx][ny]

                # Se encontramos um caminho de menor custo para a célula (nx, ny)
                if new_cost < min_cost[nx][ny]:
                    min_cost[nx][ny] = new_cost
                    queue.append((new_cost, nx, ny))

    # Se não alcançamos o destino dentro do orçamento
    return -1


In [59]:
grid = [
    [1, 3, 1],
    [1, 5, 1],
    [4, 2, 0]
]
amount = 10

print(min_cost_to_destination(grid, amount))

4
